In [ ]:
pip install keras

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import joblib
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

Import Dataset dari Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# New Section

In [ ]:
books_path = '/content/drive/My Drive/Dataset/Books.csv'
rate_path = '/content/drive/My Drive/Dataset/Ratings.csv'
user_path = '/content/drive/My Drive/Dataset/Users.csv'

books_df = pd.read_csv(books_path, error_bad_lines=False)
rate_df = pd.read_csv(rate_path, error_bad_lines=False)
user_df = pd.read_csv(user_path, error_bad_lines=False)

<ipython-input-4-b4aabd214371>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_df = pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-4-b4aabd214371>:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(books_path, error_bad_lines=False)
<ipython-input-4-b4aabd214371>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rate_df = pd.read_csv(rate_path, error_bad_lines=False)
<ipython-input-4-b4aabd214371>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user_df = pd.read_csv(user_path, error_bad_lines=False)


**JUMLAH DATA MASING-MASING ATRIBUT**

In [ ]:
print(f'Jumlah data ISBN     : {len(books_df["ISBN"].unique())}')
print(f'Jumlah data Judul    : {len(books_df["Book-Title"].unique())}')
print(f'Jumlah data Penulis  : {len(books_df["Book-Author"].unique())}')
print(f'Jumlah data Penerbit : {len(books_df["Publisher"].unique())}')
print(f'Jumlah data Tahun    : {len(books_df["Year-Of-Publication"].unique())}')
print(f'=====' * 9)
print(f'Jumlah data Pembaca              : {len(rate_df["User-ID"].unique())}')
print(f'Jumlah data Buku                 : {len(rate_df["ISBN"].unique())}')
print(f'Jumlah data Rating yang diterima : {len(rate_df)}')
print(f'=====' * 9)
print(f'Jumlah data User : {len(user_df )}')

Jumlah data ISBN     : 271360
Jumlah data Judul    : 242135
Jumlah data Penulis  : 102024
Jumlah data Penerbit : 16808
Jumlah data Tahun    : 202
Jumlah data Pembaca              : 105283
Jumlah data Buku                 : 340556
Jumlah data Rating yang diterima : 1149780
Jumlah data User : 278858


In [ ]:
books_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [ ]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [ ]:
rate_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
rate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
rate_df.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [ ]:
rate_df['Book-Rating'].describe().apply(lambda x: '%.f' % x)

count    1149780
mean           3
std            4
min            0
25%            0
50%            0
75%            7
max           10
Name: Book-Rating, dtype: object

In [ ]:
user_df

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [ ]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
user_df.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


**MENGUBAH NAMA KOLOM**

In [ ]:
books_df.rename(columns={
    'ISBN'                : 'isbn',
    'Book-Title'          : 'book_title',
    'Book-Author'         : 'book_author',
    'Year-Of-Publication' : 'pub_year',
    'Publisher'           : 'publisher',
    'Image-URL-S'         : 'image_s_url',
    'Image-URL-M'         : 'image_m_url',
    'Image-URL-L'         : 'image_l_url'
}, inplace=True)

books_df

,isbn,book_title,book_author,pub_year,publisher,image_s_url,image_m_url,image_l_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [ ]:
rate_df.rename(columns={
    'User-ID'     : 'user_id',
    'ISBN'        : 'isbn',
    'Book-Rating' : 'book_rating'
}, inplace=True)

rate_df

,user_id,isbn,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
user_df.rename(columns={
    'User-ID'  : 'user_id',
    'Location' : 'location',
    'Age'      : 'age'
}, inplace=True)

user_df

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [ ]:
ISBNAll = np.concatenate((
    books_df.isbn.unique(),
    rate_df.isbn.unique()
))

ISBNAll = np.sort(np.unique(ISBNAll))

print(f'Jumlah Buku berdasarkan ISBN : {len(ISBNAll)}')

Jumlah Buku berdasarkan ISBN : 341765


In [ ]:
UserAll = np.concatenate((
    rate_df.user_id.unique(),
    user_df.user_id.unique()
))

UserAll = np.sort(np.unique(UserAll))

print(f'Jumlah Buku berdasarkan ISBN : {len(UserAll)}')

Jumlah Buku berdasarkan ISBN : 278858


In [ ]:
books = books_df
books.isnull().sum()

isbn           0
book_title     0
book_author    1
pub_year       0
publisher      2
image_s_url    0
image_m_url    0
image_l_url    3
dtype: int64

In [ ]:
books = books.dropna()
books.isnull().sum()

isbn           0
book_title     0
book_author    0
pub_year       0
publisher      0
image_s_url    0
image_m_url    0
image_l_url    0
dtype: int64

In [ ]:
ratings = rate_df
ratings.isnull().sum()

user_id        0
isbn           0
book_rating    0
dtype: int64

In [ ]:
print(f'Total Rating 0 : {ratings.book_rating.eq(0).sum()}')

ratings = ratings[rate_df.book_rating > 0]

Total Rating 0 : 716109


In [ ]:
users = user_df
users.isnull().sum()

user_id          0
location         0
age         110762
dtype: int64

In [ ]:
users.age = users.age.fillna(users.age.mode())
users.isnull().sum()

user_id          0
location         0
age         110761
dtype: int64

**MENGECEK DATA DUPLIKAT**

In [ ]:
print(f'Jumlah data books  yang duplikat: {books.duplicated().sum()}')
print(f'Jumlah data rating yang duplikat: {ratings.duplicated().sum()}')
print(f'Jumlah data users  yang duplikat: {users.duplicated().sum()}')

Jumlah data books  yang duplikat: 0
Jumlah data rating yang duplikat: 0
Jumlah data users  yang duplikat: 0


In [ ]:
books_ratings = pd.merge(ratings, books, on=['isbn'])
books_ratings

,user_id,isbn,book_rating,book_title,book_author,pub_year,publisher,image_s_url,image_m_url,image_l_url
0,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
4,11676,038550120X,10,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...,http://images.amazon.com/images/P/038550120X.0...
...,...,...,...,...,...,...,...,...,...,...
383833,276688,0425156737,2,Graven Images,Jane Waterhouse,1997,Berkley Publishing Group,http://images.amazon.com/images/P/0425156737.0...,http://images.amazon.com/images/P/0425156737.0...,http://images.amazon.com/images/P/0425156737.0...
383834,276688,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries),Sally Gunning,1997,Pocket,http://images.amazon.com/images/P/0671563149.0...,http://images.amazon.com/images/P/0671563149.0...,http://images.amazon.com/images/P/0671563149.0...
383835,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
383836,276704,0380796155,5,White Abacus,Damien Broderick,1998,Eos,http://images.amazon.com/images/P/0380796155.0...,http://images.amazon.com/images/P/0380796155.0...,http://images.amazon.com/images/P/0380796155.0...


**MODELING**

In [ ]:
books   = books[:10000]
ratings = ratings[:5000]

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(books.book_author)
# tfidf.get_feature_names_out()

TfidfVectorizer()

In [ ]:
tfidf_matrix = tfidf.fit_transform(books.book_author)
tfidf_matrix.shape

(10000, 5575)

In [ ]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns = tfidf.get_feature_names_out(),
    index   = books.book_title
).sample(20, axis=1).sample(10, axis=0)

,nasar,tyler,fiona,marilyn,roxana,ois,crafts,dobson,warrick,weible,wladimir,seabrook,juster,winterson,suskind,cassandra,pagels,rita,foshee,emerson
book_title,,,,,,,,,,,,,,,,,,,,
The Tortilla Curtain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rite of Passage: Tales of Backpacking 'Round Europe (Lonely Planet Journeys (Travel Literature)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Bluestocking: The Story of the Famous Forrest Divorce Case,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The I Ching on Love,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tathea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Fifth Horseman : A Novel of Biological Disaster (Zammit),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cyteen III : Vindication (Cyteen),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Death Trap : An Alaska Mystery (Henry, Sue)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"MY GAL SUNDAY LARGE PRINT : \Henry and Sunday\"" Stories""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    columns = books.book_title,
    index   = books.book_title
)

print(f'Cosine Similarity Shape : {cosine_sim_df.shape}')

cosine_sim_df.sample(8, axis=1).sample(8, axis=0)


Cosine Similarity Shape : (10000, 10000)


book_title,QUEEN OF THE SUMMER STARS,Deerdancer: The Shapeshifter Archetype in Story and in Trance (Arkana S.),Naked Came the Manatee,Catcher in the Rye,The Housewife and the Assassin,The Empty House,The Wall,The Man on the Balcony: The Story of a Crime (Vintage Crime/Black Lizard)
book_title,,,,,,,,
Herr Lehmann. Ein Roman.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Shrub: The Short but Happy Political Life of George W. Bush,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kabale Und Liebe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Triggerfish Twist: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Prince,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dark Seeker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"The Know-It-All's Guide to Life: How to Climb Mount Everest, Cure Hiccups, Live to 100, and Dozens of Other Practical, Unusual, or Just Plain Fantastical Things",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Graveyard for Lunatics: Another Tale of Two Cities,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def author_recommendations_with_image(book_title, similarity_data=cosine_sim_df, items=books[['book_title', 'book_author', 'image_l_url']], k=10):
    index = similarity_data.loc[:, book_title].to_numpy().argpartition(range(-1, -k, -1))
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(book_title, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)


In [ ]:
readed_book_title = 'Proxies'

In [ ]:
books[books.book_title.eq(readed_book_title)]

,isbn,book_title,book_author,pub_year,publisher,image_s_url,image_m_url,image_l_url
50,0812523873,Proxies,Laura J. Mixon,1999,Tor Books,http://images.amazon.com/images/P/0812523873.0...,http://images.amazon.com/images/P/0812523873.0...,http://images.amazon.com/images/P/0812523873.0...


In [ ]:
recommendations_with_image = author_recommendations_with_image(readed_book_title)

In [ ]:
# Menghapus data yang terduplikasi berdasarkan 'book_title'
unique_books = books.drop_duplicates(subset=['book_title'])

# Mengurutkan DataFrame berdasarkan nama penulis (author) secara alfabetis A-Z
sorted_books = unique_books.sort_values(by='book_author', key=lambda x: x.str.lower())

# Mengambil 10 buku pertama
top_10_books = sorted_books.head(10)

# Menampilkan informasi 10 buku dengan judul, penulis, dan link
print("Rekomendasi 10 Buku Berdasarkan Penulis (Author) A-Z:\n")
for i, (index, row) in enumerate(top_10_books[['book_title', 'book_author', 'image_l_url']].iterrows(), start=1):
    book_title = row['book_title']
    book_author = row['book_author']
    image_url = row['image_l_url']

    print(f"{i}. Book Title: {book_title}")
    print(f"   Author: {book_author}")
    print(f"   Image URL: {image_url}")
    print("\n")

Rekomendasi 10 Buku Berdasarkan Penulis (Author) A-Z:

1. Book Title: On Her Own Ground : The Life and Times of Madam C.J. Walker
   Author: A'Lelia Bundles
   Image URL: http://images.amazon.com/images/P/0684825821.01.LZZZZZZZ.jpg


2. Book Title: The Dragon and the Unicorn
   Author: A. A. Attanasio
   Image URL: http://images.amazon.com/images/P/0061057797.01.LZZZZZZZ.jpg


3. Book Title: Winnie Ille Pu: A Latin Version of A. A. Milne's Winnie-The-Pooh
   Author: A. A. Milne
   Image URL: http://images.amazon.com/images/P/0525483357.01.LZZZZZZZ.jpg


4. Book Title: Pooh (Giant Board Book)
   Author: A. A. Milne
   Image URL: http://images.amazon.com/images/P/0525462325.01.LZZZZZZZ.jpg


5. Book Title: The Quest for Enlightenment: Articles from Back to Godhead Magazine
   Author: A. C. Bhaktivedanta Swami Prabhupada
   Image URL: http://images.amazon.com/images/P/0892132922.01.LZZZZZZZ.jpg


6. Book Title: Beyond Birth and Death
   Author: A. C. Bhaktivedanta Swami Prabhupada
   Imag

In [ ]:
from google.colab import files

# ... (kode program yang sudah ada)

# Save the cosine similarity matrix to a file
joblib.dump(cosine_sim_df, '/content/drive/MyDrive/cosine_similarity_matrix.pkl')

# Download the file
files.download('/content/drive/MyDrive/cosine_similarity_matrix.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>